#📌 Extracción

In [4]:
# Instalar pandas si no está disponible
!pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# Importar bibliotecas necesarias
import pandas as pd
import json

# Cargar datos desde el archivo JSON
with open('TelecomX_Data.json', 'r', encoding='utf-8') as file:
    datos_json = json.load(file)

# Mostrar la cantidad de registros cargados
print(f"Datos cargados exitosamente: {len(datos_json)} registros")
print(f"\nEstructura del primer registro:")
print(json.dumps(datos_json[0], indent=2))

Datos cargados exitosamente: 7267 registros

Estructura del primer registro:
{
  "customerID": "0002-ORFBO",
  "Churn": "No",
  "customer": {
    "gender": "Female",
    "SeniorCitizen": 0,
    "Partner": "Yes",
    "Dependents": "Yes",
    "tenure": 9
  },
  "phone": {
    "PhoneService": "Yes",
    "MultipleLines": "No"
  },
  "internet": {
    "InternetService": "DSL",
    "OnlineSecurity": "No",
    "OnlineBackup": "Yes",
    "DeviceProtection": "No",
    "TechSupport": "Yes",
    "StreamingTV": "Yes",
    "StreamingMovies": "No"
  },
  "account": {
    "Contract": "One year",
    "PaperlessBilling": "Yes",
    "PaymentMethod": "Mailed check",
    "Charges": {
      "Monthly": 65.6,
      "Total": "593.3"
    }
  }
}


In [6]:
# Convertir JSON a DataFrame normalizando los datos anidados
df = pd.json_normalize(datos_json)

# Mostrar informacion basica del DataFrame
print(f"Dimensiones del DataFrame: {df.shape[0]} filas x {df.shape[1]} columnas")
print(f"\nColumnas disponibles:")
print(df.columns.tolist())
print(f"\nPrimeros 5 registros:")
df.head()

Dimensiones del DataFrame: 7267 filas x 21 columnas

Columnas disponibles:
['customerID', 'Churn', 'customer.gender', 'customer.SeniorCitizen', 'customer.Partner', 'customer.Dependents', 'customer.tenure', 'phone.PhoneService', 'phone.MultipleLines', 'internet.InternetService', 'internet.OnlineSecurity', 'internet.OnlineBackup', 'internet.DeviceProtection', 'internet.TechSupport', 'internet.StreamingTV', 'internet.StreamingMovies', 'account.Contract', 'account.PaperlessBilling', 'account.PaymentMethod', 'account.Charges.Monthly', 'account.Charges.Total']

Primeros 5 registros:


,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


#🔧 Transformación

In [7]:
# ============================================
# 2.1 Exploracion de la estructura del dataset
# ============================================

# Informacion general del DataFrame
print("INFORMACION GENERAL DEL DATASET")
print("=" * 50)
df.info()

print("\n\nTIPOS DE DATOS POR COLUMNA")
print("=" * 50)
print(df.dtypes)

INFORMACION GENERAL DEL DATASET
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7267 non-null   object 
 1   Churn                      7267 non-null   object 
 2   customer.gender            7267 non-null   object 
 3   customer.SeniorCitizen     7267 non-null   int64  
 4   customer.Partner           7267 non-null   object 
 5   customer.Dependents        7267 non-null   object 
 6   customer.tenure            7267 non-null   int64  
 7   phone.PhoneService         7267 non-null   object 
 8   phone.MultipleLines        7267 non-null   object 
 9   internet.InternetService   7267 non-null   object 
 10  internet.OnlineSecurity    7267 non-null   object 
 11  internet.OnlineBackup      7267 non-null   object 
 12  internet.DeviceProtection  7267 non-null   object 
 13  internet.TechSup

In [8]:
# ============================================
# 2.2 Identificacion de problemas en los datos
# ============================================

# Valores ausentes por columna
print("VALORES AUSENTES POR COLUMNA")
print("=" * 50)
valores_nulos = df.isnull().sum()
print(valores_nulos[valores_nulos > 0] if valores_nulos.sum() > 0 else "No hay valores nulos")

# Valores vacios (strings vacios o espacios)
print("\n\nVALORES VACIOS (STRINGS VACIOS)")
print("=" * 50)
for col in df.select_dtypes(include=['object']).columns:
    vacios = (df[col] == '') | (df[col].str.strip() == '')
    if vacios.sum() > 0:
        print(f"{col}: {vacios.sum()} valores vacios")

# Duplicados
print("\n\nREGISTROS DUPLICADOS")
print("=" * 50)
duplicados = df.duplicated().sum()
print(f"Total de registros duplicados: {duplicados}")

# Duplicados por customerID
duplicados_id = df['customerID'].duplicated().sum()
print(f"IDs de cliente duplicados: {duplicados_id}")

# Estadisticas descriptivas
print("\n\nESTADISTICAS DESCRIPTIVAS (VARIABLES NUMERICAS)")
print("=" * 50)
df.describe()

VALORES AUSENTES POR COLUMNA
No hay valores nulos


VALORES VACIOS (STRINGS VACIOS)
Churn: 224 valores vacios
account.Charges.Total: 11 valores vacios


REGISTROS DUPLICADOS
Total de registros duplicados: 0
IDs de cliente duplicados: 0


ESTADISTICAS DESCRIPTIVAS (VARIABLES NUMERICAS)


,customer.SeniorCitizen,customer.tenure,account.Charges.Monthly
count,7267.000000,7267.000000,7267.000000
mean,0.162653,32.346498,64.720098
std,0.369074,24.571773,30.129572
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.425000
50%,0.000000,29.000000,70.300000
75%,0.000000,55.000000,89.875000
max,1.000000,72.000000,118.750000


In [9]:
# ============================================
# 2.3 Limpieza y correccion de datos
# ============================================

# Crear copia para trabajar
df_limpio = df.copy()

# Verificar y corregir la columna de cargos totales (puede estar como string)
if df_limpio['account.Charges.Total'].dtype == 'object':
    # Convertir a numerico, reemplazando valores no convertibles con NaN
    df_limpio['account.Charges.Total'] = pd.to_numeric(df_limpio['account.Charges.Total'], errors='coerce')
    print("Columna 'account.Charges.Total' convertida a numerico")

# Verificar valores nulos despues de la conversion
nulos_total = df_limpio['account.Charges.Total'].isnull().sum()
print(f"Valores nulos en 'account.Charges.Total' despues de conversion: {nulos_total}")

# Rellenar valores nulos con la mediana (si existen)
if nulos_total > 0:
    mediana = df_limpio['account.Charges.Total'].median()
    df_limpio['account.Charges.Total'].fillna(mediana, inplace=True)
    print(f"Valores nulos rellenados con la mediana: {mediana}")

# Eliminar duplicados si existen
duplicados_antes = len(df_limpio)
df_limpio = df_limpio.drop_duplicates()
duplicados_eliminados = duplicados_antes - len(df_limpio)
print(f"\nDuplicados eliminados: {duplicados_eliminados}")

# Verificar el resultado
print(f"\nDataset limpio: {df_limpio.shape[0]} filas x {df_limpio.shape[1]} columnas")

Columna 'account.Charges.Total' convertida a numerico
Valores nulos en 'account.Charges.Total' despues de conversion: 11
Valores nulos rellenados con la mediana: 1391.0

Duplicados eliminados: 0

Dataset limpio: 7267 filas x 21 columnas


C:\Users\Usuario\AppData\Local\Temp\ipykernel_48016\329572436.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_limpio['account.Charges.Total'].fillna(mediana, inplace=True)


In [10]:
# ============================================
# 2.4 Crear columna Cuentas_Diarias
# ============================================

# Calcular el valor diario basado en la facturacion mensual
# Asumiendo 30 dias por mes
df_limpio['Cuentas_Diarias'] = df_limpio['account.Charges.Monthly'] / 30

print("Columna 'Cuentas_Diarias' creada exitosamente")
print("\nEstadisticas de Cuentas_Diarias:")
print(df_limpio['Cuentas_Diarias'].describe())

print("\nMuestra de las columnas de facturacion:")
df_limpio[['account.Charges.Monthly', 'account.Charges.Total', 'Cuentas_Diarias']].head(10)

Columna 'Cuentas_Diarias' creada exitosamente

Estadisticas de Cuentas_Diarias:
count    7267.000000
mean        2.157337
std         1.004319
min         0.608333
25%         1.180833
50%         2.343333
75%         2.995833
max         3.958333
Name: Cuentas_Diarias, dtype: float64

Muestra de las columnas de facturacion:


,account.Charges.Monthly,account.Charges.Total,Cuentas_Diarias
0,65.60,593.30,2.186667
1,59.90,542.40,1.996667
2,73.90,280.85,2.463333
3,98.00,1237.85,3.266667
4,83.90,267.40,2.796667
5,69.40,571.45,2.313333
6,109.70,7904.25,3.656667
7,84.65,5377.80,2.821667
8,48.20,340.35,1.606667
9,90.45,5957.90,3.015000


In [11]:
# ============================================
# 2.5 Estandarizacion y renombrado de columnas
# ============================================

# Diccionario de mapeo para renombrar columnas
mapeo_columnas = {
    'customerID': 'id_cliente',
    'Churn': 'cancelacion',
    'customer.gender': 'genero',
    'customer.SeniorCitizen': 'es_adulto_mayor',
    'customer.Partner': 'tiene_pareja',
    'customer.Dependents': 'tiene_dependientes',
    'customer.tenure': 'meses_contrato',
    'phone.PhoneService': 'servicio_telefonico',
    'phone.MultipleLines': 'lineas_multiples',
    'internet.InternetService': 'servicio_internet',
    'internet.OnlineSecurity': 'seguridad_online',
    'internet.OnlineBackup': 'respaldo_online',
    'internet.DeviceProtection': 'proteccion_dispositivo',
    'internet.TechSupport': 'soporte_tecnico',
    'internet.StreamingTV': 'streaming_tv',
    'internet.StreamingMovies': 'streaming_peliculas',
    'account.Contract': 'tipo_contrato',
    'account.PaperlessBilling': 'factura_electronica',
    'account.PaymentMethod': 'metodo_pago',
    'account.Charges.Monthly': 'cargo_mensual',
    'account.Charges.Total': 'cargo_total',
    'Cuentas_Diarias': 'cargo_diario'
}

# Renombrar columnas
df_limpio = df_limpio.rename(columns=mapeo_columnas)

print("Columnas renombradas exitosamente")
print("\nNuevas columnas:")
print(df_limpio.columns.tolist())

Columnas renombradas exitosamente

Nuevas columnas:
['id_cliente', 'cancelacion', 'genero', 'es_adulto_mayor', 'tiene_pareja', 'tiene_dependientes', 'meses_contrato', 'servicio_telefonico', 'lineas_multiples', 'servicio_internet', 'seguridad_online', 'respaldo_online', 'proteccion_dispositivo', 'soporte_tecnico', 'streaming_tv', 'streaming_peliculas', 'tipo_contrato', 'factura_electronica', 'metodo_pago', 'cargo_mensual', 'cargo_total', 'cargo_diario']


In [12]:
# ============================================
# 2.6 Convertir valores Yes/No a binarios (1/0)
# ============================================

# Columnas con valores Yes/No
columnas_binarias = [
    'cancelacion', 'tiene_pareja', 'tiene_dependientes',
    'servicio_telefonico', 'factura_electronica'
]

# Mapeo de valores
mapeo_binario = {'Yes': 1, 'No': 0}

for col in columnas_binarias:
    if col in df_limpio.columns:
        df_limpio[col] = df_limpio[col].map(mapeo_binario)
        print(f"Columna '{col}' convertida a binario")

# Verificar la conversion
print("\nVerificacion de valores unicos en columnas binarias:")
for col in columnas_binarias:
    if col in df_limpio.columns:
        print(f"{col}: {df_limpio[col].unique()}")

Columna 'cancelacion' convertida a binario
Columna 'tiene_pareja' convertida a binario
Columna 'tiene_dependientes' convertida a binario
Columna 'servicio_telefonico' convertida a binario
Columna 'factura_electronica' convertida a binario

Verificacion de valores unicos en columnas binarias:
cancelacion: [ 0.  1. nan]
tiene_pareja: [1 0]
tiene_dependientes: [1 0]
servicio_telefonico: [1 0]
factura_electronica: [1 0]


In [13]:
# ============================================
# 2.7 Resumen del dataset transformado
# ============================================

print("RESUMEN DEL DATASET TRANSFORMADO")
print("=" * 50)
print(f"Total de registros: {len(df_limpio)}")
print(f"Total de columnas: {len(df_limpio.columns)}")

print("\n\nINFORMACION DEL DATASET FINAL")
print("=" * 50)
df_limpio.info()

print("\n\nMUESTRA DE LOS PRIMEROS REGISTROS")
print("=" * 50)
df_limpio.head()

RESUMEN DEL DATASET TRANSFORMADO
Total de registros: 7267
Total de columnas: 22


INFORMACION DEL DATASET FINAL
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id_cliente              7267 non-null   object 
 1   cancelacion             7043 non-null   float64
 2   genero                  7267 non-null   object 
 3   es_adulto_mayor         7267 non-null   int64  
 4   tiene_pareja            7267 non-null   int64  
 5   tiene_dependientes      7267 non-null   int64  
 6   meses_contrato          7267 non-null   int64  
 7   servicio_telefonico     7267 non-null   int64  
 8   lineas_multiples        7267 non-null   object 
 9   servicio_internet       7267 non-null   object 
 10  seguridad_online        7267 non-null   object 
 11  respaldo_online         7267 non-null   object 
 12  proteccion_dispositivo  7267 non-n

,id_cliente,cancelacion,genero,es_adulto_mayor,tiene_pareja,tiene_dependientes,meses_contrato,servicio_telefonico,lineas_multiples,servicio_internet,...,proteccion_dispositivo,soporte_tecnico,streaming_tv,streaming_peliculas,tipo_contrato,factura_electronica,metodo_pago,cargo_mensual,cargo_total,cargo_diario
0,0002-ORFBO,0.0,Female,0,1,1,9,1,No,DSL,...,No,Yes,Yes,No,One year,1,Mailed check,65.6,593.30,2.186667
1,0003-MKNFE,0.0,Male,0,0,0,9,1,Yes,DSL,...,No,No,No,Yes,Month-to-month,0,Mailed check,59.9,542.40,1.996667
2,0004-TLHLJ,1.0,Male,0,0,0,4,1,No,Fiber optic,...,Yes,No,No,No,Month-to-month,1,Electronic check,73.9,280.85,2.463333
3,0011-IGKFF,1.0,Male,1,1,0,13,1,No,Fiber optic,...,Yes,No,Yes,Yes,Month-to-month,1,Electronic check,98.0,1237.85,3.266667
4,0013-EXCHZ,1.0,Female,1,1,0,3,1,No,Fiber optic,...,No,Yes,Yes,No,Month-to-month,1,Mailed check,83.9,267.40,2.796667


#📊 Carga y análisis

#📄Informe final